### **Segunda limpieza del dataset**

- 1. Reconfigurar PRODUCTO/idProducto (PKs).
- 2. Mapear DETALLE_VENTA/idProducto con PRODUCTO/idProducto.
- 3. Agregar atributos estándar a todas las tablas.
- 4. Exportar dataframes a excel.

In [1]:
import pandas as pd

# Leer archivos .xls <pip install xlrd>
df_venta = pd.read_excel('01_ventas.xls', sheet_name="VENTA")
df_producto = pd.read_excel('01_ventas.xls', sheet_name="PRODUCTO")
df_detalle_venta = pd.read_excel('01_ventas.xls', sheet_name="DETALLE_VENTA")

---

#### 1. Reconfigurar PRODUCTO/idProducto (PKs)
- Prefijo: primeras tres letras de la columna *categoria* en mayúscula.
- Sufijo: valor autoincremental (001, 002, ...) para cada producto dentro de la categoría.

In [2]:
df_producto[["idProducto", "nombre", "categoria"]].head()

,idProducto,nombre,categoria
0,1,Medialuna,Dulces
1,2,Latte,Cafés
2,3,Chipá,Salados
3,4,Flat White,Cafés
4,5,Onda Medialunas,Promos


In [3]:
# Generar IDs de 3 cifras por cada productos dentro de una categoría
df_producto["id_nuevo"] = (
    df_producto.groupby("categoria").cumcount() + 1
).astype(str).str.zfill(3)

# Generar prefijo en mayúscula con las primeras 3 letras de la categoría
df_producto["id_nuevo"] = df_producto["categoria"].str[:3].str.upper() + df_producto["id_nuevo"]

# Cambiar los valores de idProducto por los nuevos
df_producto["idProducto"] = df_producto["id_nuevo"]
df_producto = df_producto.drop(columns=["id_nuevo"])

df_producto[["idProducto", "nombre", "categoria"]].head(10)

,idProducto,nombre,categoria
0,DUL001,Medialuna,Dulces
1,CAF001,Latte,Cafés
2,SAL001,Chipá,Salados
3,CAF002,Flat White,Cafés
4,PRO001,Onda Medialunas,Promos
5,DUL002,Cookie Red Velvet,Dulces
6,DUL003,Cookie Clásica,Dulces
7,DUL004,Pan de Chocolate,Dulces
8,CAF003,Magic,Cafés
9,DUL005,Croissant,Dulces


---

#### 2. Mapear *DETALLE_VENTA/idProducto* con *PRODUCTO/idProducto*

- Eliminar información redundante.
- Relacionar ambas tablas.

In [4]:
df_detalle_venta[["idVenta","producto","categoria"]].head()

,idVenta,producto,categoria
0,6812,Latte,Café con leche
1,6812,Pan de Chocolate,Dulces
2,6812,Vaso 12oz,Adicionales
3,6811,Cookie Clásica,Cookies
4,6811,Alfacookie,Alfajores


In [5]:
# Crear mapa de nombres a IDs
mapa = dict(zip(df_producto["nombre"], df_producto["idProducto"]))

# Reemplazar en detalle_venta
df_detalle_venta["idProducto"] = df_detalle_venta["producto"].map(mapa)

# Eliminar columnas reemplazadas
df_detalle_venta = df_detalle_venta.drop(columns=["producto", "categoria"])

# Ordenar columnas
df_detalle_venta = df_detalle_venta[["idVenta", "idProducto", "cantidad", "precio", "costo", "cancelada"]]

df_detalle_venta[["idVenta", "idProducto"]].head()

,idVenta,idProducto
0,6812,CAF001
1,6812,DUL004
2,6812,ADI001
3,6811,DUL003
4,6811,DUL009


---

#### 3. Agregar atributos estándar a todas las tablas
- creacion: fecha de creación de registro
- actualizacion: fecha de actualización de registro
- activo: posibilidad de borrado lógico

In [6]:
# Estandarizar PRODUCTO
df_producto["creacion"] = df_producto["actualizacion"] = pd.Timestamp.now().replace(microsecond=0)
df_producto["activo"] = True

df_producto.head()

,idProducto,nombre,categoria,cantidad,total_ARS,creacion,actualizacion,activo
0,DUL001,Medialuna,Dulces,2487,3091300,2025-09-19 08:28:59,2025-09-19 08:28:59,True
1,CAF001,Latte,Cafés,1412,5261410,2025-09-19 08:28:59,2025-09-19 08:28:59,True
2,SAL001,Chipá,Salados,1382,3498725,2025-09-19 08:28:59,2025-09-19 08:28:59,True
3,CAF002,Flat White,Cafés,933,3539800,2025-09-19 08:28:59,2025-09-19 08:28:59,True
4,PRO001,Onda Medialunas,Promos,721,4103500,2025-09-19 08:28:59,2025-09-19 08:28:59,True


In [7]:
# Estandarizar VENTA
df_venta["creacion"] = df_venta["actualizacion"] = df_venta["hora"]
df_venta = df_venta.drop(columns=["fecha", "hora"])
df_venta["activo"] = True

df_venta.head()

,idVenta,total,tipo,creacion,actualizacion,activo
0,6812,10700.0,Mostrador,2025-09-04 19:14:06,2025-09-04 19:14:06,True
1,6811,8300.0,Mostrador,2025-09-04 19:11:49,2025-09-04 19:11:49,True
2,6810,7100.0,Mostrador,2025-09-04 19:10:58,2025-09-04 19:10:58,True
3,6809,4500.0,Mostrador,2025-09-04 19:07:29,2025-09-04 19:07:29,True
4,6808,0.0,Local,2025-09-04 19:04:18,2025-09-04 19:04:18,True


In [8]:
# Estandarizar DETALLE_VENTA
df_detalle_venta_aux = df_detalle_venta.merge(
    df_venta[["idVenta", "creacion"]],
    on="idVenta",
    how="left"
)

df_detalle_venta["creacion"] = df_detalle_venta["actualizacion"] = df_detalle_venta_aux["creacion"]

df_detalle_venta["activo"] = True

df_detalle_venta.head()

,idVenta,idProducto,cantidad,precio,costo,cancelada,creacion,actualizacion,activo
0,6812,CAF001,1,4200,1155.47,No,2025-09-04 19:14:06,2025-09-04 19:14:06,True
1,6812,DUL004,1,5000,2250.00,No,2025-09-04 19:14:06,2025-09-04 19:14:06,True
2,6812,ADI001,1,1500,174.00,No,2025-09-04 19:14:06,2025-09-04 19:14:06,True
3,6811,DUL003,1,3800,1100.00,No,2025-09-04 19:11:49,2025-09-04 19:11:49,True
4,6811,DUL009,1,4500,1434.00,No,2025-09-04 19:11:49,2025-09-04 19:11:49,True


---

#### 4. Exportar dataframes a excel
- 01_ventas.xls -> pandas.df -> 02_ventas.xlsx

In [9]:
with pd.ExcelWriter("02_ventas.xlsx") as writer:
    df_venta.to_excel(writer, sheet_name="VENTA", index=False)
    df_detalle_venta.to_excel(writer, sheet_name="DETALLE_VENTA", index=False)
    df_producto.to_excel(writer, sheet_name="PRODUCTO", index=False)

df_venta.to_csv("02_venta.csv", index=False)
df_detalle_venta.to_csv("02_detalle_venta.csv", index=False)
df_producto.to_csv("02_producto.csv", index=False)